In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Declare some properties
num_classes = 7
batch_size = 256
epochs = 50

In [7]:
# Load data
with open('fer2013/fer2013.csv') as f:
    content = f.readlines()

lines = np.array(content)
num_of_instances = lines.size

In [10]:
# Train test set split
x_train, y_train, x_test, y_test = [], [], [], []

for i in range(1, num_of_instances):
    emotion, img, usage = lines[i].split(',')
    val = img.split(' ')
    pixels = np.array(val, 'float32')
    emotion = tensorflow.keras.utils.to_categorical(emotion, num_classes)
    
    if 'Training' in usage:
        y_train.append(emotion)
        x_train.append(pixels)
    elif 'PublicTest' in usage:
        y_test.append(emotion)
        x_test.append(pixels)

In [12]:
# Convert to array
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')
x_train /= 255  # normalize inputs between [0, 1]
x_test /= 255

# Check shape
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(28709, 2304)
(28709, 7)
(3589, 2304)
(3589, 7)


In [13]:
# Convert pixels to shape 48x48x1
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)

In [14]:
# Create model
model = Sequential()
model.add(Conv2D(64, (5, 5), activation = 'relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size = (5, 5), strides = (2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
# Make batch generator
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size = batch_size)

# Model compile
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [18]:
# Train model
model.fit_generator(train_generator, steps_per_epoch = (batch_size), epochs = epochs)

Epoch 1/50
256/256 [==============================] - 68s 266ms/step - loss: 1.8071 - acc: 0.2513
Epoch 2/50
256/256 [==============================] - 20s 80ms/step - loss: 1.6062 - acc: 0.3572
Epoch 3/50
256/256 [==============================] - 20s 80ms/step - loss: 1.4319 - acc: 0.4472
Epoch 4/50
256/256 [==============================] - 20s 80ms/step - loss: 1.3169 - acc: 0.4951
Epoch 5/50
256/256 [==============================] - 20s 79ms/step - loss: 1.2231 - acc: 0.5326
Epoch 6/50
256/256 [==============================] - 20s 78ms/step - loss: 1.1598 - acc: 0.5602
Epoch 7/50
256/256 [==============================] - 20s 79ms/step - loss: 1.0944 - acc: 0.5852
Epoch 8/50
256/256 [==============================] - 20s 78ms/step - loss: 1.0370 - acc: 0.6076
Epoch 9/50
256/256 [==============================] - 20s 78ms/step - loss: 0.9752 - acc: 0.6333
Epoch 10/50
256/256 [==============================] - 20s 77ms/step - loss: 0.9122 - acc: 0.6572
Epoch 11/50
256/256 [=======

In [19]:
# Save model
model.save('emo_model.h5')

In [20]:
# Overall evaluation
score = model.evaluate(x_test, y_test)

print('Test loss: ', score[0])
print('Test accuracy: ', 100*score[1])

3589/3589 [==============================] - 1s 301us/sample - loss: 3.9421 - acc: 0.5631
Test loss:  3.9420589937011856
Test accuracy:  56.310951709747314
